<a href="https://colab.research.google.com/github/meryltheng/DeepLearningA-Z/blob/main/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

Image augmentation - a type of transformation to obtain new images.

In [ ]:
train_datagen = ImageDataGenerator( #transformation
    rescale=1./255, # pixel values are divided by 255; normalisation
    shear_range=0.2, # feature scaling params from here
    zoom_range=0.2,
    horizontal_flip=True)
training_set = train_datagen.flow_from_directory( # connect images from your training set
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

### Preprocessing the Test set

Note that we do not apply the same transformations here, keep the original test images!!
We need to feature scale the test images to match the scale of the training set.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

First convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, # no of feature detectors
                               kernel_size=3, # dimensions of feature detector
                               activation='relu',
                               input_shape=(64,64,3))) # last arg is 3 correspond to RGB; 1 if b/w

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, # size of pooling frame
                                  strides=2 # shift frame every 2 px
                                  ))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, # no of feature detectors
                               kernel_size=3, # dimensions of feature detector
                               activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, # size of pooling frame
                                  strides=2 # shift frame every 2 px
                                  ))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layer.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, # more hidden neurons compared to prev example
                              activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, # 1 neuron to encode binary info (cat or dog)
                              activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Note the difference with training the previous ANN on only the training set.

## Part 4 - Making a single prediction

Before we make a prediction, validation image needs to be in the right format (i.e., matches that in the cnn).
Single image has to be in a batch - cnn model has to regonise the batch as an extra dimension.

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # so that dimension of the batch will be the 1st dimension
result = cnn.predict(test_image/255.0)
training_set.class_indices # check what 0 and 1 correspond to - dog or cat
if result[0][0] >0.5: # first [] is the batch; second [] is the batch element
  prediction = 'dog'
else:
  prediction = 'cat'

Note: index in Python starts from 0.

In [ ]:
print(prediction)